## Navigate to the Correct Directory

The following code navigates to the dataprocessing directory.

In [12]:
cd ../dataprocessing

/usr/local/google/home/carverforbes/activity-recognition/location/lbs/activity/audioset/dataprocessing


## Call the import statements

The following code imports the necessary code to run the code in the rest of this notebook.

In [24]:
# import statements
import audio_processing as ap

import functools
import os
import sys
from absl import logging
import csv

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow import feature_column
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import sklearn

# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
# tf.keras.backend.set_floatx('float32')

print("Ran the import statements.")

Ran the import statements.


In [14]:
# set logging to print logging.INFO logs
logging.set_verbosity(logging.INFO)

In [203]:
# arguments for audio_processing
src_dir = 'example_src_dir'
dest_dir = 'example_dest_dir'
filename = 'testset'
labels = ['Gunshot, gunfire']
available_features = ['chroma_stft',
                       'chroma_cqt',
                       'chroma_cens',
                       'melspectrogram',
                       'mfcc',
                       'rms',
                       'spectral_centroid',
                       'spectral_bandwidth',
                       'spectral_contrast',
                       'spectral_flatness',
                       'spectral_rolloff',
                       'poly_features',
                       'tonnetz',
                       'zero_crossing_rate'
]
features_to_extract = ['rms']
last_features = None
redo = False

In [204]:
def dataframe_to_csv(dataframe, dest_path):
    df = dataframe.copy()
    for column in df.columns:   
        for i in range(df[column].size):
            if isinstance(df[column][i], np.ndarray):
                df[column][i] = df[column][i].tolist()
    df.to_csv(path, index=False, header=True)

In [205]:
def csv_to_dataframe(csv_path):
    df = pd.read_csv(csv_path)
    for column in df.columns:
        for i in range(df[column].size):
            if isinstance(df[column][i], str):
                temp = df[column][i][1:-1]
                temp = temp.replace(',', ' ').split(']')
                new_list = []
                for item in temp:
                    if item == '':
                        continue
                    item = item.replace('[', '').strip().split()
                    item = [float(num) for num in item]
                    arr = np.array(item)
                    new_list.append(arr)
                new_arr = np.array(new_list)
                df[column][i] = new_arr
    return df

In [ ]:
# Import the dataset as a pandas Dataframe object.
csv_path = os.path.join(dest_dir, filename + '.csv')
features_changed = last_features is None or features_to_extract != last_features
last_features = features_to_extract
if os.path.isfile(csv_path) and not True and not features_changed:
    df = csv_to_dataframe(csv_path)
else:
    df = ap.output_df(src_dir, dest_dir, filename, labels, features_to_extract, redo)
    dataframe_to_csv(df, csv_path)

In [210]:
df.head()

,label,rms
0,1,"[[0.024051616, 0.02332899, 0.023757856, 0.0242..."
1,1,"[[0.018282544, 0.018028082, 0.017703537, 0.016..."
2,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,1,"[[0.04337968, 0.04965238, 0.052755404, 0.05585..."
4,0,"[[0.0066741644, 0.00669201, 0.0066719446, 0.00..."


In [207]:
print(len(df['rms'][4]))
print(type(df['rms'][0]))
print(type(df['rms'][0][0][0]))
print(type(df['label'][0]))

1
<class 'numpy.ndarray'>
<class 'numpy.float32'>
<class 'numpy.int64'>


In [208]:
df_from_csv = csv_to_dataframe(csv_path)

<ipython-input-205-014a385941ca>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column][i] = new_arr


In [209]:
df_from_csv.head()

,label,rms
0,1,"[[0.024051615968346596, 0.0233289897441864, 0...."
1,1,"[[0.018282543867826462, 0.018028082326054573, ..."
2,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,1,"[[0.04337967932224274, 0.049652379006147385, 0..."
4,0,"[[0.006674164440482855, 0.006692009977996349, ..."


In [212]:
print(len(df_from_csv['rms'][9]))
print(type(df_from_csv['rms'][9]))
print(type(df_from_csv['rms'][9][0][0]))
print(type(df_from_csv['label'][9]))

1
<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'numpy.int64'>


In [195]:
train_df_rms = df_from_csv.copy()
train_df_rms.head()

,label,melspectrogram
0,1,"[[3.4128592014312744, 2.5363638401031494, 2.29..."
1,1,"[[0.009717190638184547, 0.002868133829906583, ..."
2,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,1,"[[0.17820563912391663, 0.09990730881690979, 0...."
4,0,"[[0.0009625130333006382, 0.0006070901872590184..."


In [196]:
train_df_rms = train_df_rms.reindex(
    np.random.permutation(train_df_rms.index))
train_df_rms.head()

,label,melspectrogram
6,0,"[[0.052792735397815704, 0.03404252976179123, 0..."
26,1,"[[0.4272644519805908, 0.3823623061180115, 0.67..."
9,1,"[[1.435829520225525, 1.8679133653640747, 1.284..."
22,1,"[[0.043675199151039124, 0.015942996367812157, ..."
23,0,"[[3505.867431640625, 3847.041015625, 4128.8388..."


In [197]:
# temp bug fix for rows with None features
# also deletes the outer array in the features
rows_with_none = []
print(train_df_rms.size / 2)
# print(train_df_rms.rms[0])
for i in train_df_rms.index:
    if train_df_rms.rms[i] is None:
        rows_with_none.append(i)
        continue
    train_df_rms.rms[i] = train_df_rms.rms[i][0]
print(rows_with_none)
train_df_rms = train_df_rms.drop(rows_with_none)
train_df_rms.size / 2
train_df_rms.head(10000)

29.0


AttributeError: 'DataFrame' object has no attribute 'rms'

In [ ]:
X = np.array(train_df_rms.rms.tolist(), dtype=object)
y = np.array(train_df_rms.label.tolist())

In [ ]:
# Temp bug fix for having different sized features
temp_x = []
temp_y = []
count = 0
for arr, label in zip(X, y):
    if arr is None or len(arr) < 431:
#         print('hey look at me', len(arr))
        count += 1
        continue
    temp_x.append(arr)
    temp_y.append(label)
X = np.array(temp_x, dtype=object)
y = np.array(temp_y)
print(count)

In [ ]:
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
print(y_val)
print(y_train)

In [ ]:
# trying to fix bug:
# ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).
# It worked!!!
from keras import backend as K
x_train = K.cast_to_floatx(x_train)
y_train = K.cast_to_floatx(y_train)
x_val = K.cast_to_floatx(x_val)
y_val = K.cast_to_floatx(y_val)

In [ ]:
# Define the plotting function.
def plot_curve(epochs, hist, list_of_metrics, path, filename, list_of_hyperparameters):
    """Plot a curve of one or more classification metrics vs. epoch and save it to path."""  
    # list_of_metrics should be one of the names shown in:
    # https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#define_the_model_and_metrics  

    plt.figure()
    plt.xlabel("Epoch")
    plt.ylabel("Value")

    for m in list_of_metrics:
        x = hist[m]
        plt.plot(epochs[1:], x[1:], label=m)

    plt.legend()
    
    if not os.path.isdir(path):
        try:
            os.mkdir(path)
        except OSError as error:
            logging.error(error)
            
    path = os.path.join(path, filename)
    
    if not os.path.isdir(path):
        try:
            os.mkdir(path)
        except OSError as error:
            logging.error(error)
        
    list_of_hyperparameters_temp = [str(item) for item in list_of_hyperparameters]
    list_of_metrics_temp = [item if isinstance(item, str) else str(item.name) for item in list_of_metrics]
    filename = '_'.join(list_of_metrics_temp) + '_' + '_'.join(list_of_hyperparameters_temp)
    path = os.path.join(path, filename + '.png')
    plt.savefig(path, bbox_inches='tight')
    
    return plt


print("Defined the plot_curve function.")

## Train 0
Deep Neural Network:
* Input Layer: 431 nodes
* Hidden Layer 1: 431 nodes
* Ouput Layer: 1 node

Hyper-parameters:
* Loss Function: BinaryCrossEntropy
* Activation Function: Relu
* Optimizer Function: RMSprop
* Learning Rate: 0.001 
* Epochs: 100
* Batch_Size: 25
* Classification Threshold: 0.7
* Regularization: L2
* Regularization Lambda: 0.05

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of 431 nodes
    model.add(tf.keras.layers.Dense(units=431, input_shape=(431,)))
              
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=431, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.05),
                                  name='Hidden1'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.0,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")  

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 100
batch_size = 25
classification_threshold = 0.70
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           label_name]

# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS)

my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot, 'example_dest_dir/training_rms_dnn/', filename, list_of_hyperparameters)
plot_curve(epochs, hist, ['loss', 'val_loss'], 'example_dest_dir/training_rms_dnn/', filename, list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

## Train 1
Deep Neural Network:
* Input Layer: 431 nodes
* Hidden Layer 1: 431 nodes
* Hidden Layer 2: 40 nodes
* Ouput Layer: 1 node

Hyper-parameters:
* Loss Function: BinaryCrossEntropy
* Activation Function: Relu
* Optimizer Function: RMSprop
* Learning Rate: 0.001 
* Epochs: 250
* Batch_Size: 25
* Classification Threshold: 0.7
* Regularization: L2
* Regularization Lambda: 0.05

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of 431 nodes
    model.add(tf.keras.layers.Dense(units=431, input_shape=(431,)))
              
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=431, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.05),
                                  name='Hidden1'))
    
    # Implement L2 regularization in the second hidden layer.
    model.add(tf.keras.layers.Dense(units=40, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.05),
                                  name='Hidden2'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.1,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")  

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.001
epochs = 250
batch_size = 25
classification_threshold = 0.70
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           label_name]

# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS)

my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot, 'example_dest_dir/training_rms_dnn/', filename, list_of_hyperparameters)
plot_curve(epochs, hist, ['loss', 'val_loss'], 'example_dest_dir/training_rms_dnn/', filename, list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])

## Train 2
Deep Neural Network:
* Input Layer: 431 nodes
* Hidden Layer 1: 431 nodes
* Hidden Layer 2: 40 nodes
* Hidden Layer 3: 20 nodes
* Ouput Layer: 1 node

Hyper-parameters:
* Loss Function: BinaryCrossEntropy
* Activation Function: Relu
* Optimizer Function: RMSprop
* Learning Rate: 0.001 
* Epochs: 200
* Batch_Size: 25
* Classification Threshold: 0.7
* Regularization: L2
* Regularization Lambda: 0.05

In [ ]:
# Define the functions that create and train a model.
def create_model(my_learning_rate, my_metrics):
    """Create and compile a simple classification model."""
    # Discard any pre-existing version of the model.
    model = None

    # Most simple tf.keras models are sequential.
    model = tf.keras.models.Sequential()

    # Add the input layer of 431 nodes
    model.add(tf.keras.layers.Dense(units=431, input_shape=(431,)))
              
    # Implement L2 regularization in the first hidden layer.
    model.add(tf.keras.layers.Dense(units=431, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.05),
                                  name='Hidden1'))
    
    # Implement L2 regularization in the second hidden layer.
    model.add(tf.keras.layers.Dense(units=40, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.05),
                                  name='Hidden2'))
    
    # Implement L2 regularization in the third hidden layer.
    model.add(tf.keras.layers.Dense(units=20, 
                                  activation='relu',
                                  kernel_regularizer=tf.keras.regularizers.l2(0.05),
                                  name='Hidden3'))

    # Funnel the regression value through a sigmoid function.
    model.add(tf.keras.layers.Dense(units=1, input_shape=(1,),
                                  activation=tf.sigmoid,
                                  name='Output'))

    # Call the compile method to construct the layers into a model that
    # TensorFlow can execute.  Notice that we're using a different loss
    # function for classification than for regression.    
    model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),                                                   
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=my_metrics)

    return model        
              
def train_model(model, features, label, epochs, label_name,
                batch_size=None, my_validation_split=0.1,
                validation_data=None, shuffle=True):
    """Feed a dataset into the model in order to train it."""

    # The x parameter of tf.keras.Model.fit can be a list of arrays.
    history = model.fit(x=features, y=label, batch_size=batch_size,
                      epochs=epochs, shuffle=shuffle, validation_data=validation_data)

    # The list of epochs is stored separately from the rest of history.
    epochs = history.epoch

    # Isolate the classification metric for each epoch.
    hist = pd.DataFrame(history.history)

    return epochs, hist  

print("Defined the create_model and train_model functions.")  

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.002
epochs = 200
batch_size = 50
classification_threshold = 0.70
label_name = "label"

list_of_hyperparameters = [learning_rate, epochs, batch_size,
                           classification_threshold,
                           label_name]

# Here is the updated definition of METRICS:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(
          name='accuracy', threshold=classification_threshold),
      tf.keras.metrics.Precision(
          thresholds=classification_threshold, name='precision'),
      tf.keras.metrics.Recall(
          thresholds=classification_threshold, name="recall"),
]

# Establish the model's topography.
my_model = create_model(learning_rate, METRICS)

# Print the models structure
my_model.summary()

# Train the model on the training set.
epochs, hist = train_model(my_model, x_train, y_train, epochs, 
                          label_name, batch_size, validation_data=(x_val, y_val))

# Plot metrics vs. epochs
list_of_metrics_to_plot = ['accuracy', "precision", "recall", 'val_accuracy', 'val_precision', 'val_recall'] 
plot_curve(epochs, hist, list_of_metrics_to_plot, 'example_dest_dir/training_rms_dnn/', filename, list_of_hyperparameters)
plot_curve(epochs, hist, ['loss', 'val_loss'], 'example_dest_dir/training_rms_dnn/', filename, list_of_hyperparameters)

training_performance =  my_model.evaluate(x_train, y_train, verbose=0)
print('Training Performance')
print('---------------------------------')
print('loss: ', training_performance[0])
print('accuracy: ', training_performance[1])
print('precision: ', training_performance[2])
print('recall: ', training_performance[3])
print()

validation_performance =  my_model.evaluate(x_val, y_val, verbose=0)
print('Validation Performance')
print('---------------------------------')
print('loss: ', validation_performance[0])
print('accuracy: ', validation_performance[1])
print('precision: ', validation_performance[2])
print('recall: ', validation_performance[3])